<a href="https://colab.research.google.com/github/fmigone/App_Codification_Python_CAE/blob/main/Camembert_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Préambule

In [ ]:
#Installation des packages
!pip install transformers

In [ ]:
#Importation de packages
import re
import json
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from transformers import (
    CamembertTokenizer,
    TFCamembertModel,
    TFRobertaPreTrainedModel,
    TFRobertaMainLayer
)
from transformers.modeling_tf_utils import get_initializer
from tensorflow.keras.layers import *

In [ ]:
#Hyper_Parameter
BATCH_SIZE = np.array([16,24,32])
STEPS = 32
EPOCHS = 10
LR = np.array([1e-05,2e-05,3e-05])
LR_DECAY = "Linear"
WD = 0.01
WR = 0.06
Adam_epsilon = 1e-6 
Adam_beta = 0.9 
Adam_beta = 0.98 

## Preparation des données

In [ ]:
#Chargement des données
df = pd.read_excel("/content/drive/MyDrive/Base_CITI_Produits_REVU.xlsx")

In [ ]:
#Traitement
df = df[['Libelle', 'CODE']]
df['CODE'] = df['CODE'].astype(int)
num_classes = len(df['CODE'].value_counts())

## Import Pretrained Model end Embeddings

## Tokenization et Embeddings

### Tokenization

### Embeddings

## Modelisation

In [26]:
import inspect

In [ ]:
TFRobertaForSequenceClassification

In [35]:
source_DF = inspect.getsource(TFRobertaForSequenceClassification)
print(source_DF)

class TFRobertaForSequenceClassification(TFRobertaPreTrainedModel, TFSequenceClassificationLoss):
    # names with a '.' represents the authorized unexpected/missing layers when a TF model is loaded from a PT model
    _keys_to_ignore_on_load_unexpected = [r"pooler", r"lm_head"]

    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.roberta = TFRobertaMainLayer(config, add_pooling_layer=False, name="roberta")
        self.classifier = TFRobertaClassificationHead(config, name="classifier")

    @unpack_inputs
    @add_start_docstrings_to_model_forward(ROBERTA_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        processor_class=_TOKENIZER_FOR_DOC,
        checkpoint="cardiffnlp/twitter-roberta-base-emotion",
        output_type=TFSequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output="'optimism'",
        e

In [34]:
from transformers import TFRobertaForSequenceClassification,TFCamembertForSequenceClassification, CamembertTokenizer

ImportError: ignored

In [19]:
class TFCamembertForSequenceClassification(TFRobertaPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.roberta = TFRobertaMainLayer(config, name="roberta")
        self.classifier = TFRobertaClassificationHead(config, name="classifier")

    def call(self, inputs, **kwargs):
        outputs = self.roberta(inputs, **kwargs)
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output, training=kwargs.get("training", False))
        outputs = (logits,) + outputs[2:]
        return outputs  # logits, (hidden_states), (attentions)

In [ ]:
model = TFCamembertForSequenceClassification.from_pretrained("jplu/tf-camembert-base",num_labels=num_classes)
tokenizer = CamembertTokenizer.from_pretrained(
    "jplu/tf-camembert-base",
    output_hidden_states=True,
    output_attentions=True
)

### Installation du modèle pré-entrainé

### Recherche des Hyper-Paramètres

## Entrainement

## Exportation